In [1]:
# resnet

In [2]:
from transformers import AutoFeatureExtractor, ResNetModel

/home/yizhou/miniconda3/envs/cliport/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from datasets import load_dataset

In [4]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

No config specified, defaulting to: cats-image/image
Reusing dataset cats-image (/home/yizhou/.cache/huggingface/datasets/huggingface___cats-image/image/1.9.0/68fbc793fb10cd165e490867f5d61fa366086ea40c73e549a020103dcb4f597e)
100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 828.59it/s]


In [5]:
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-18")
model = ResNetModel.from_pretrained("microsoft/resnet-18")

Some weights of the model checkpoint at microsoft/resnet-18 were not used when initializing ResNetModel: ['classifier.1.weight', 'classifier.1.bias']
- This IS expected if you are initializing ResNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ResNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
inputs = feature_extractor(image, return_tensors="pt")

In [7]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [8]:
with torch.no_grad():
    outputs = model(**inputs)

In [9]:
outputs.last_hidden_state.shape

torch.Size([1, 512, 7, 7])

In [10]:
from custom_cliport import *

In [11]:
# Clip

In [13]:
from transformers import CLIPTokenizer, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

inputs = tokenizer(["a photo of a cat"], padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs)

In [14]:
l = text_features

In [15]:
x = outputs.last_hidden_state

In [16]:
cc = CustomCliport()

In [17]:
x.shape, l.shape

(torch.Size([1, 512, 7, 7]), torch.Size([1, 512]))

In [18]:
x = cc(x, l)

In [19]:
x.shape

torch.Size([1, 1, 224, 224])

In [20]:
x = F.interpolate(x, size=(256, 256), mode='bilinear')

In [21]:
x.shape

torch.Size([1, 1, 256, 256])

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [23]:
count_parameters(cc)

2587633

In [25]:
ALL_SEMANTIC_TYPES = [f"{v_desc}_{h_desc}_{cabinet_type}" for v_desc in ["", "bottom", "second-bottom", "middle", "second-top", "top"] for h_desc in ["", "right", "second-right", "middle", "second-left", "left"] for cabinet_type in ["drawer", "door"]]

In [31]:
all_texts = [t.replace("_"," ").replace("-"," ").replace("  ", " ").strip() for t in ALL_SEMANTIC_TYPES]

In [46]:
# all_texts

In [ ]:
from transformers import CLIPTokenizer, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

In [33]:
inputs = tokenizer(all_texts, padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs)

In [38]:
text2feature = {all_texts[i]: text_features[i].data for i in range(72)}

In [40]:
torch.sum((text2feature['drawer'] - text2feature['right drawer'])**2)

tensor(6.3948)

In [45]:
torch.sum((text2feature['drawer'] - text2feature['middle door'])**2)

tensor(43.5977)

In [48]:
import pickle
# save dictionary to pickle file
with open('text2clip_feature.pickle', 'wb') as file:
    pickle.dump(text2feature, file, protocol=pickle.HIGHEST_PROTOCOL)